<h1 style="
  font-size: 55px;
  text-align: center;
  font-family: 'Poppins', sans-serif;
  text-shadow: 2px 2px 5px rgba(63, 155, 77, 0.84);
  letter-spacing: 2px;
">
  <b>📊 Merged Dataset</b>
</h1>
<hr style="width: 60%; border: 2px solid #21996fd1; border-radius: 5px;">


## <span style="color:#21996fd1;">**6- Model Training & Evaluation**
,---

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, f1_score, classification_report

from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import TomekLinks

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [ ]:
df=pd.read_csv("encoded_data.csv")

In [ ]:
y = df["TARGET"]
X = df.drop(columns=["TARGET"])
# Use the representative sample created above

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    stratify=y,
    random_state=42
)

print("\nTrain distribution:", Counter(y_train))
print("Test distribution :", Counter(y_test))


In [ ]:
numeric_cols = X_train.select_dtypes(include=["float64", "int64"]).columns

scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])


In [ ]:
def best_threshold(y_true, y_proba):
    thresholds = np.arange(0.05, 0.90, 0.01)
    best_f1 = 0
    best_t = 0.5

    for t in thresholds:
        preds = (y_proba >= t).astype(int)
        score = f1_score(y_true, preds)
        if score > best_f1:
            best_f1 = score
            best_t = t

    return best_t, best_f1


In [ ]:
def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    print(f"\n===================== {name} =====================")
    model.fit(X_train, y_train)

    proba_test = model.predict_proba(X_test)[:, 1]

    # ----------- NEW: find best threshold ----------
    t, f1 = best_threshold(y_test, proba_test)
    print(f"Best threshold: {t:.2f} | Best F1: {f1:.4f}")

    # Predict with best threshold
    pred_test = (proba_test >= t).astype(int)

    print("Test AUC:", roc_auc_score(y_test, proba_test))
    print("Test F1 :", f1_score(y_test, pred_test))
    print("\nClassification Report:")
    print(classification_report(y_test, pred_test))


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
sample_size = 40000
X_sample = X_train.sample(sample_size, random_state=42)
y_sample = y_train.loc[X_sample.index]

rf_params = {
    "n_estimators": [200, 400, 600, 900],
    "max_depth": [6, 10, 15, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False]
}

rf_search = RandomizedSearchCV(
    RandomForestClassifier(),
    rf_params,
    n_iter=25,
    scoring="f1",
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

rf_search.fit(X_sample, y_sample)
print(rf_search.best_params_)


In [ ]:
xgb_params = {
    "n_estimators": [300, 500, 700],
    "learning_rate": [0.02, 0.04, 0.06],
    "max_depth": [3, 4, 5, 6],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.6, 0.7, 0.8],
    "gamma": [0, 1, 5]
}

xgb_search = RandomizedSearchCV(
    XGBClassifier(eval_metric="logloss"),
    xgb_params,
    cv=3,
    n_iter=25,
    n_jobs=-1,
    scoring="f1",
    verbose=1,
    random_state=42
)

xgb_search.fit(X_sample, y_sample)
print(xgb_search.best_params_)


In [ ]:
lgb_params = {
    "num_leaves": [20, 30, 40, 60],
    "learning_rate": [0.01, 0.02, 0.03, 0.04],
    "min_child_samples": [20, 50, 100, 150],
    "feature_fraction": [0.6, 0.7, 0.8, 0.9],
    "bagging_fraction": [0.6, 0.7, 0.8],
    "bagging_freq": [3, 5, 10],
    "reg_alpha": [0, 1, 5],
    "reg_lambda": [0, 2, 5]
}

lgb_search = RandomizedSearchCV(
    lgb.LGBMClassifier(),
    lgb_params,
    n_iter=30,
    scoring="f1",
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

lgb_search.fit(X_sample, y_sample)
print(lgb_search.best_params_)


In [ ]:
cat_params = {
    "depth": [4, 5, 6, 7, 8],
    "learning_rate": [0.02, 0.03, 0.04],
    "l2_leaf_reg": [1, 3, 5, 7, 9],
    "bagging_temperature": [0.3, 0.5, 1]
}

cat_search = RandomizedSearchCV(
    CatBoostClassifier(verbose=False),
    cat_params,
    n_iter=20,
    scoring="f1",
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

cat_search.fit(X_sample, y_sample)
print(cat_search.best_params_)


In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        bootstrap=False,
        random_state=42
    ),
    "XGBoost": XGBClassifier(
        n_estimators=700,
        learning_rate=0.06,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.7,
        eval_metric="logloss",
        random_state=42,
        gamma=0,
    ),
    "LightGBM": lgb.LGBMClassifier(
        n_estimators=400,
        learning_rate=0.04,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),
    "CatBoost": CatBoostClassifier(
        iterations=400,
        learning_rate=0.04,
        depth=6,
        verbose=False,
        random_state=42
    )
}
for name, model in models.items():
    evaluate_model(name, model, X_train, y_train, X_test, y_test)



In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# =========================================================
# 1) Train Random Forest on balanced training data
# =========================================================

rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)

rf.fit(X_train, y_train)

# =========================================================
# 2) Extract feature importance
# =========================================================

importances = rf.feature_importances_
feature_names = X_train.columns

feat_imp = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values(by="importance", ascending=False)

print("\nTop 40 most important features:\n")
print(feat_imp.head(40))

# =========================================================
# 3) Plot Top 25 Features (optional)
# =========================================================

plt.figure(figsize=(10, 14))
plt.barh(feat_imp.head(25)["feature"], feat_imp.head(25)["importance"])
plt.gca().invert_yaxis()
plt.title("Top 25 Important Features - Random Forest")
plt.show()


In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score, roc_auc_score, classification_report

# ===============================
# FOCAL PROBABILITY TRANSFORM
# ===============================
def focal_sigmoid(p, gamma=2.0):
    return p ** gamma   # simulate focal focusing

# ===============================
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'learning_rate': 0.006,
    'num_leaves': 31,
    'max_depth': 7,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 200,
    'min_split_gain': 0.01,
    'reg_alpha': 5.0,
    'reg_lambda': 10.0,
    'class_weight': 'balanced',
    'subsample_for_bin': 200000,
}



model = lgb.LGBMClassifier(
    **params,
    n_estimators=3000
)

# ===============================
# FIT
# ===============================
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)


# ===============================
# PREDICT WITH FOCAL
# ===============================
raw_pred = model.predict_proba(X_test)[:, 1]
focal_pred = focal_sigmoid(raw_pred)

best_f1, best_thr = 0, 0
for thr in np.linspace(0.1, 0.6, 50):
    pred = (focal_pred >= thr).astype(int)
    f1 = f1_score(y_test, pred)
    if f1 > best_f1:
        best_f1, best_thr = f1, thr

print(f"\nBest threshold: {best_thr:.4f}")
print(f"Best F1: {best_f1:.4f}")

final_pred = (focal_pred >= best_thr).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, final_pred))

print("AUC:", roc_auc_score(y_test, focal_pred))

# ===============================
raw_pred_train = model.predict_proba(X_train)[:, 1]
focal_pred_train = focal_sigmoid(raw_pred_train)

train_pred = (focal_pred_train >= best_thr).astype(int)

print("\n=========== TRAIN REPORT ===========")
print(classification_report(y_train, train_pred))
print("Train AUC:", roc_auc_score(y_train, focal_pred_train))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# حساب الـ confusion matrix
cm = confusion_matrix(y_test, final_pred)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np
import lightgbm as lgb


def focal_sigmoid(p, gamma=3.0):
    return p ** gamma


params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'learning_rate': 0.006,
    'num_leaves': 31,
    'max_depth': 7,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 200,
    'min_split_gain': 0.01,
    'reg_alpha': 5.0,
    'reg_lambda': 10.0,
    'class_weight': 'balanced',
    'subsample_for_bin': 200000,
    'metric': 'binary_logloss'
}

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_auc = []
fold_f1 = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):

    print(f"\n==================== Fold {fold+1} ====================")

    X_tr, X_val = X.iloc[train_idx], X.iloc[valid_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[valid_idx]

    model = lgb.LGBMClassifier(
        **params,
        n_estimators=3000
    )

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(stopping_rounds=100)]
    )

    raw = model.predict_proba(X_val)[:, 1]
    focal = focal_sigmoid(raw)

    # find best threshold per fold
    best_f1, best_thr = 0, 0
    for thr in np.linspace(0.05, 0.5, 80):
        pred = (focal >= thr).astype(int)
        f1 = f1_score(y_val, pred)
        if f1 > best_f1:
            best_f1, best_thr = f1, thr

    fold_auc.append(roc_auc_score(y_val, focal))
    fold_f1.append(best_f1)

    print(f"Fold AUC: {fold_auc[-1]:.4f}")
    print(f"Fold F1 : {best_f1:.4f} @ thr={best_thr:.4f}")


print("\n==================== FINAL CV RESULT ====================")
print(f"Mean AUC: {np.mean(fold_auc):.4f}  |  Std: {np.std(fold_auc):.4f}")
print(f"Mean F1 : {np.mean(fold_f1):.4f}  |  Std: {np.std(fold_f1):.4f}")

In [ ]:
# =====================================================
# 1) IMPORTS
# =====================================================
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras import backend as K   # <-- IMPORTANT FIX
from sklearn.metrics import roc_auc_score, f1_score, classification_report


# =====================================================
# 2) FOCAL LOSS FUNCTION (Correct Version)
# =====================================================
def focal_loss(gamma=2., alpha=0.25):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, dtype='float32')
        y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())  # numerical stability

        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)

        weight = alpha * K.pow((1 - y_pred), gamma)
        return K.mean(weight * cross_entropy)
    return loss


# =====================================================
# 3) BUILD THE MODEL
# =====================================================
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    layers.Dense(32, activation='relu'),
    layers.Dropout(0.1),

    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=focal_loss(gamma=2, alpha=0.25),
    metrics=['AUC']
)


# =====================================================
# 4) TRAIN WITH EARLY STOPPING
# =====================================================
es = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_split=0.15,
    epochs=25,
    batch_size=512,
    callbacks=[es],
    verbose=1
)


# =====================================================
# 5) EVALUATE
# =====================================================
y_train_proba = model.predict(X_train).ravel()
y_test_proba = model.predict(X_test).ravel()

y_train_pred = (y_train_proba > 0.5).astype(int)
y_test_pred = (y_test_proba > 0.5).astype(int)

print("\n================ NEURAL NETWORK WITH FOCAL LOSS ================")
print("Train AUC:", roc_auc_score(y_train, y_train_proba))
print("Test AUC:", roc_auc_score(y_test, y_test_proba))
print("Train F1:", f1_score(y_train, y_train_pred))
print("Test F1:", f1_score(y_test, y_test_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

In [ ]:
df.head()

# model

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, classification_report

# SELECT ONLY THE TOP 17 FEATURES
# =========================================================
top_17_features = [
    "EXT_SOURCE_2",
    "EXT_SOURCE_3",
    "ANNUITY_CREDIT_RATIO",
    "ANNUITY_INCOME_RATIO",
    "CREDIT_INCOME_RATIO",
    "YEARS_EMPLOYED",
    "APPLICATION_CREDIT_RATIO_MEAN",
    "APPLICATION_CREDIT_RATIO_MAX",
    "APPLICATION_CREDIT_RATIO_STD",
    "CREDIT_GOODS_RATIO_MEAN",
    "CREDIT_GOODS_RATIO_MAX",
    "CREDIT_GOODS_RATIO_STD",
    "APPLICATION_CREDIT_RATIO_MIN",
    "CREDIT_GOODS_RATIO_MIN",
    "SOCIAL_CIRCLE_AVG",
    "AMT_REQ_CREDIT_BUREAU_YEAR",
    "CREDIT_3M_TO_LAST_RATIO_MEAN"
]

# DROP EVERYTHING EXCEPT 17 FEATURES + TARGET
# =========================================================
df = df[top_17_features + ["TARGET"]]

# SPLIT TRAIN / TEST
# =========================================================
X = df[top_17_features]
y = df["TARGET"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

# SHOW TARGET DISTRIBUTION
print("\n=== ORIGINAL TARGET DISTRIBUTION ===")
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

# FOCAL TRANSFORMATION
def focal_sigmoid(p, gamma=2.0):
    return p ** gamma

# LIGHTGBM PARAMETERS
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'learning_rate': 0.006,
    'num_leaves': 31,
    'max_depth': 7,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 200,
    'min_split_gain': 0.01,
    'reg_alpha': 5.0,
    'reg_lambda': 10.0,
    'class_weight': 'balanced',
}

model = lgb.LGBMClassifier(
    **params,
    n_estimators=3000
)

# TRAIN MODEL
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)

# PREDICT RAW PROBABILITY
raw_pred = model.predict_proba(X_test)[:, 1]

print("\n=== VALUE COUNTS BEFORE FOCAL (RAW PROB>=0.5) ===")
print(pd.Series((raw_pred >= 0.5).astype(int)).value_counts(normalize=True))

# APPLY FOCAL TRANSFORM
focal_pred = focal_sigmoid(raw_pred)

print("\n=== VALUE COUNTS AFTER FOCAL (>=0.5) ===")
print(pd.Series((focal_pred >= 0.5).astype(int)).value_counts(normalize=True))

# FIND BEST THRESHOLD
best_f1, best_thr = 0, 0
for thr in np.linspace(0.1, 0.6, 50):
    pred = (focal_pred >= thr).astype(int)
    f1 = f1_score(y_test, pred)
    if f1 > best_f1:
        best_f1, best_thr = f1, thr

print(f"\nBest threshold: {best_thr:.4f}")
print(f"Best F1: {best_f1:.4f}")

# FINAL PREDICTION
final_pred = (focal_pred >= best_thr).astype(int)

print("\n=== VALUE COUNTS AFTER THRESHOLD OPTIM ===")
print(pd.Series(final_pred).value_counts(normalize=True))

# REPORTS
print("\nClassification Report:")
print(classification_report(y_test, final_pred))

print("AUC:", roc_auc_score(y_test, focal_pred))

# TRAIN EVAL
raw_pred_train = model.predict_proba(X_train)[:, 1]
focal_pred_train = focal_sigmoid(raw_pred_train)
train_pred = (focal_pred_train >= best_thr).astype(int)

print("\n=========== TRAIN REPORT ===========")
print(classification_report(y_train, train_pred))
print("Train AUC:", roc_auc_score(y_train, focal_pred_train))


In [ ]:
import os
import pickle

os.makedirs("models", exist_ok=True)

with open("models/credit_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("\nModel saved successfully as models/credit_model.pkl")


In [ ]:
for col in df.columns:
  print(col.upper())
  print(df[col].unique())
  print("\n--------------------------------------")

In [ ]:
print(X_train.columns.tolist())


In [ ]:
df['TARGET'].value_counts(normalize=True)

In [ ]:
df.drop(columns=["TARGET"]).head(30).to_csv("sample_head.csv", index=False)


In [ ]:
df.info()

# unsupervised

In [ ]:
label_col = "TARGET"
X = df.drop(columns=[label_col]).values
y = df[label_col].astype(int).values

from sklearn.model_selection import train_test_split

# 80% Train — 20% Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)


In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn import metrics

contamination_est = max(0.001, y.mean())
print("Estimated contamination:", contamination_est)

iso = IsolationForest(
    n_estimators=100,
    contamination=contamination_est,
    random_state=42,
    n_jobs=-1
)

iso.fit(X_train)
y_pred_test = (iso.predict(X_test) == -1).astype(int)

print(metrics.classification_report(y_test, y_pred_test))


In [ ]:
import optuna
from sklearn import metrics

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int("n_estimators", 50, 400),
        'max_samples': trial.suggest_float("max_samples", 0.5, 1.0),
        'max_features': trial.suggest_float("max_features", 0.5, 1.0),
        'bootstrap': trial.suggest_categorical("bootstrap", [True, False]),
        'contamination': trial.suggest_float("contamination", contamination_est*0.5, contamination_est*1.5),
    }

    model = IsolationForest(
        **params,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train)

    preds = (model.predict(X_test) == -1).astype(int)
    f1 = metrics.f1_score(y_test, preds, zero_division=0)

    return f1

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40)

print("Best Params →", study.best_params)
print("Best F1 →", study.best_value)


In [ ]:
best_params = study.best_params

best_model = IsolationForest(
    **best_params,
    random_state=42,
    n_jobs=-1
)

# Train على كل الـ Train data فقط
best_model.fit(X_train)

y_pred_test_best = (best_model.predict(X_test) == -1).astype(int)

print(metrics.classification_report(y_test, y_pred_test_best))

iso_score_train = -best_model.score_samples(X_train)
iso_score_test = -best_model.score_samples(X_test)

X_train_with_iso = np.column_stack([X_train, iso_score_train])
X_test_with_iso = np.column_stack([X_test, iso_score_test])

y_pred_test_best = (best_model.predict(X_test) == -1).astype(int)

print("IsolationForest classification report (before using as feature):")
print(metrics.classification_report(y_test, y_pred_test_best))

print("New shapes after adding iso_score:")
print(X_train_with_iso.shape, X_test_with_iso.shape)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# Negative score_samples → higher = more anomaly
scores = -best_model.score_samples(X_test)

# ROC
fpr, tpr, _ = roc_curve(y_test, scores)
roc_auc = auc(fpr, tpr)

# PR
precision, recall, _ = precision_recall_curve(y_test, scores)

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()

plt.subplot(1,2,2)
plt.plot(recall, precision)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")

plt.show()


In [ ]:
thresholds = np.linspace(min(scores), max(scores), 200)
best_f1 = -1
best_t = None

for t in thresholds:
    preds = (scores >= t).astype(int)   # 1 anomaly
    f1 = metrics.f1_score(y_test, preds, zero_division=0)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best Threshold =", best_t)
print("Best F1 after threshold tuning =", best_f1)


In [ ]:
y_pred_threshold = (scores >= best_t).astype(int)

print(metrics.classification_report(y_test, y_pred_threshold))
